# Proyecto BBDD

## Importamos librerías

In [24]:
import os
import sqlite3
from datetime import datetime
import pandas as pd

import psycopg2

print("Librerías importadas correctamente")

Librerías importadas correctamente


## Importamos y limpiamos los CSV

In [26]:
# Importamos los csv con los datos del ejercicio
df_alumnos_1= pd.read_csv('Data/Clase_1.csv', sep= ";")
df_alumnos_2= pd.read_csv('Data/Clase_2.csv', sep= ";")
df_alumnos_3= pd.read_csv('Data/Clase_3.csv', sep= ";")
df_alumnos_4= pd.read_csv('Data/Clase_4.csv', sep= ";")
df_profesores= pd.read_csv('Data/Claustro.csv', sep= ";")

In [ ]:
# Unimos los df de alumnos en uno solo con el que poder trabajar

df_alumnos= pd.concat([df_alumnos_1, df_alumnos_2, df_alumnos_3, df_alumnos_4], ignore_index= True)
df_alumnos.head(5)

,Nombre,Email,Promoción,Fecha_comienzo,Campus,Proyecto_HLF,Proyecto_EDA,Proyecto_BBDD,Proyecto_ML,Proyecto_Deployment,Proyecto_WebDev,Proyecto_FrontEnd,Proyecto_Backend,Proyecto_React,Proyecto_FullSatck
0,Jafet Casals,Jafet_Casals@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto,NaN,NaN,NaN,NaN,NaN
1,Jorge Manzanares,Jorge_Manzanares@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto,NaN,NaN,NaN,NaN,NaN
2,Onofre Adadia,Onofre_Adadia@gmail.com,Septiembre,18/09/2023,Madrid,Apto,Apto,Apto,No Apto,Apto,NaN,NaN,NaN,NaN,NaN
3,Merche Prada,Merche_Prada@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,No Apto,Apto,No Apto,NaN,NaN,NaN,NaN,NaN
4,Pilar Abella,Pilar_Abella@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Cambiamos el nombre de las verticales para que no den error más adelante

df_profesores['Vertical'] = df_profesores['Vertical'].replace({
    'DS': 'Data Science',
    'FS': 'Full Stack'
})

## Conectamos el Notebook a la base de datos

In [ ]:
# Creamos la conexión al servidor de Render

connection = psycopg2.connect(
        host= 'dpg-d41069hr0fns739r6gq0-a.oregon-postgres.render.com',
        database= 'bootcamp_z1q7',
        user= 'bootcamp_z1q7_user',
        password= '4MDOTwmhpav1upPCxzDtPrNStbgNmb2X',
        port= 5432
        ) 

cursor = connection.cursor()

In [ ]:
# Con pgAdmin 4 vinculado a Render modificamos las tablas
# Comprobación de que funciona:

cursor.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public';
""")
tablas = cursor.fetchall()
print("Tablas en la base de datos:")
for tabla in tablas:
    print(tabla[0])

Tablas en la base de datos:
Alumnos
Campus
Vertical
Promo
Clases
Profesores
Matriculas
Asignaciones
Expediente


## Rellenamos tablas

#### Tabla: Alumnos

In [34]:
# Rellenamos cada alumno con nombre e email con un bucle

for index, row in df_alumnos.iterrows():
    cursor.execute('''
    INSERT INTO "Alumnos" ("Nombre", "email")
    VALUES (%s, %s)
    ''', (
        row['Nombre'], 
        row['Email']
    ))

connection.commit()

#### Tabla: Profesores

In [40]:
# Rellenamos cada alumno con nombre e email con un bucle

for index, row in df_profesores.iterrows():
    cursor.execute('''
    INSERT INTO "Profesores" ("Nombre")
    VALUES (%s)
    ''', (
        row['Nombre'],
    ))

connection.commit()

#### Tabla: Asignaciones

In [55]:
# Ahora toca tirar algo de código ChatGPT para conseguir hacer coincidir la info de los profes del DF con la tabla 'Clases' ya rellenada

for index, row in df_profesores.iterrows():
    nombre = row['Nombre']
    vertical = row['Vertical']
    campus = row['Campus']
    promocion = row['Promocion']
    rol = row['Rol']
    modalidad = row['Modalidad']

    cursor.execute('SELECT "ID_Profe" FROM "Profesores" WHERE "Nombre" = %s', (nombre,))
    id_profe = cursor.fetchone()

    cursor.execute('''
        SELECT "ID_Clase"
        FROM "Clases"
        JOIN "Vertical" ON "Clases"."ID_Vertical" = "Vertical"."ID_Vertical"
        JOIN "Campus" ON "Clases"."ID_Campus" = "Campus"."ID_Campus"
        JOIN "Promo" ON "Clases"."ID_Promo" = "Promo"."ID_Promo"
        WHERE "Vertical"."Vertical" = %s AND "Campus"."Ciudad" = %s AND "Promo"."Promocion" = %s
    ''', (vertical, campus, promocion))
    id_clase = cursor.fetchone()

    cursor.execute('''
        INSERT INTO "Asignaciones" ("ID_Profe", "ID_Clase", "Rol", "Modalidad")
        VALUES (%s, %s, %s, %s)
    ''', (id_profe[0], id_clase[0], rol, modalidad))

connection.commit()

In [ ]:
# Este pedazo del código ha servido para ver que combiancion de clases de profes frente a alumnos no existía,
# lo que daba un eror en la celda anterior

missing = 0
for i, row in df_profesores.iterrows():
    cursor.execute('''SELECT COUNT(*) FROM "Clases" c
                      JOIN "Vertical" v ON c."ID_Vertical" = v."ID_Vertical"
                      JOIN "Campus" ca ON c."ID_Campus" = ca."ID_Campus"
                      JOIN "Promo" p ON c."ID_Promo" = p."ID_Promo"
                      WHERE v."Vertical"=%s AND ca."Ciudad"=%s AND p."Promocion"=%s''',
                      (row['Vertical'], row['Campus'], row['Promocion']))
    if cursor.fetchone()[0] == 0:
        missing += 1

print(f"Clases no encontradas: {missing}")

Clases no encontradas: 0


#### Tabla: Matrículas

In [ ]:
# Tiramos el mismo código pero adaptádolo a alumnos -> matriculas <- clases

for index, row in df_alumnos.iterrows():
    nombre = row['Nombre']
    campus = row['Campus']
    promocion = row['Promoción']

    # Recemos porque esto funcione
    if row['Proyecto_HLF'] == 'Apto' or row['Proyecto_HLF'] == 'No Apto':
        vertical = 'Data Science'
    else: vertical = 'Full Stack'

    cursor.execute('SELECT "ID_Alumno" FROM "Alumnos" WHERE "Nombre" = %s', (nombre,))
    id_alumno = cursor.fetchone()

    cursor.execute('''
        SELECT "ID_Clase"
        FROM "Clases"
        JOIN "Vertical" ON "Clases"."ID_Vertical" = "Vertical"."ID_Vertical"
        JOIN "Campus" ON "Clases"."ID_Campus" = "Campus"."ID_Campus"
        JOIN "Promo" ON "Clases"."ID_Promo" = "Promo"."ID_Promo"
        WHERE "Vertical"."Vertical" = %s AND "Campus"."Ciudad" = %s AND "Promo"."Promocion" = %s
    ''', (vertical, campus, promocion))
    id_clase = cursor.fetchone()

    cursor.execute('''
        INSERT INTO "Matriculas" ("ID_Alumno", "ID_Clase")
        VALUES (%s, %s)
    ''', (id_alumno[0], id_clase[0]))

connection.commit()

#### Tablas: Clases, Vertical, Campus, Promo

Estas tablas las hemos rellenado usando pgAdmin 4.  
Las tablas Vertical, Campus y Promo manualmente por ser sencillas.  
La tabla clases con el siguiente query:

INSERT INTO "Clases" ("ID_Vertical", "ID_Campus", "ID_Promo")  
VALUES  
(  
(select "ID_Vertical" from "Vertical" where "Vertical"= 'Data Science'),  
(select "ID_Campus" from "Campus" where "Ciudad"= 'Madrid'),  
(select "ID_Promo" from "Promo" where "Promocion"= 'Septiembre')  
),  
  
(  
(select "ID_Vertical" from "Vertical" where "Vertical"= 'Data Science'),  
(select "ID_Campus" from "Campus" where "Ciudad"= 'Madrid'),  
(select "ID_Promo" from "Promo" where "Promocion"= 'Febrero')  
),   
  
(  
(select "ID_Vertical" from "Vertical" where "Vertical"= 'Full Stack'),  
(select "ID_Campus" from "Campus" where "Ciudad"= 'Madrid'),   
(select "ID_Promo" from "Promo" where "Promocion"= 'Septiembre')  
),  
  
(  
(select "ID_Vertical" from "Vertical" where "Vertical"= 'Full Stack'),  
(select "ID_Campus" from "Campus" where "Ciudad"= 'Valencia'),  
(select "ID_Promo" from "Promo" where "Promocion"= 'Febrero')  
)  
  
(  
(select "ID_Vertical" from "Vertical" where "Vertical"= 'Full Stack'),  
(select "ID_Campus" from "Campus" where "Ciudad"= 'Valencia'),  
(select "ID_Promo" from "Promo" where "Promocion"= 'Septiembre')  
),  
  
(  
(select "ID_Vertical" from "Vertical" where "Vertical"= 'Full Stack'),  
(select "ID_Campus" from "Campus" where "Ciudad"= 'Madrid'),  
(select "ID_Promo" from "Promo" where "Promocion"= 'Febrero')  
);  

## Rollback

Si da error, tirar de aquí, cuando pueda lo implemento en un try except

In [61]:
# SI ALGO FALLA EJECUTAR ESTO ANTES DE SEGUIR
connection.rollback()

## Preparamos algunas queries para tirar

Las voy a dejar listas con un 'formato' pgAdmin 4:

#### Alumnos que estudian cursan Data Science

SELECT  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Alumnos"."Nombre",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Vertical"."Vertical",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Campus"."Ciudad",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Promo"."Promocion"  
FROM "Alumnos"  
JOIN "Matriculas" ON "Alumnos"."ID_Alumno" = "Matriculas"."ID_Alumno"  
JOIN "Clases" ON "Matriculas"."ID_Clase" = "Clases"."ID_Clase"  
JOIN "Vertical" ON "Clases"."ID_Vertical" = "Vertical"."ID_Vertical"  
JOIN "Campus" ON "Clases"."ID_Campus" = "Campus"."ID_Campus"  
JOIN "Promo" ON "Clases"."ID_Promo" = "Promo"."ID_Promo"  
WHERE "Vertical"."Vertical" = 'Data Science'  

#### Profesores que imparten clase en cursos relacionados con el campus de Valencia

SELECT  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Profesores"."Nombre",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Vertical"."Vertical",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Campus"."Ciudad",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Promo"."Promocion",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Asignaciones"."Modalidad",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Asignaciones"."Rol"  
FROM "Profesores"  
JOIN "Asignaciones" ON "Profesores"."ID_Profe" = "Asignaciones"."ID_Profe"  
JOIN "Clases" ON "Asignaciones"."ID_Clase" = "Clases"."ID_Clase"  
JOIN "Vertical" ON "Clases"."ID_Vertical" = "Vertical"."ID_Vertical"  
JOIN "Campus" ON "Clases"."ID_Campus" = "Campus"."ID_Campus"  
JOIN "Promo" ON "Clases"."ID_Promo" = "Promo"."ID_Promo"  
WHERE "Campus"."Ciudad" = 'Valencia'  

#### Calificaciones de los proyectos para los estudiantes de Data Science

SELECT  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Alumnos"."Nombre",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."HLF",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."EDA",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."BBDD",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."ML",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."Deployment"
FROM "Alumnos"  
JOIN "Matriculas" ON "Matriculas"."ID_Alumno" = "Alumnos"."ID_Alumno"  
JOIN "Expediente" ON "Expediente"."ID_Matricula" = "Matriculas"."ID_Matricula"  
JOIN "Clases" ON "Clases"."ID_Clase" = "Matriculas"."ID_Matricula"  
JOIN "Vertical" ON "Vertical"."ID_Vertical" = "Clases"."ID_Vertical"  
WHERE "Vertical"."Vertical" = 'Data Science'  

#### Alumnos de Full Stack que hayan aprobado todos sus proyectos

SELECT  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Alumnos"."Nombre",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."Web Dev",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."Front End",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."Back End",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."React",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"Expediente"."Full Stack"  
FROM "Alumnos"  
JOIN "Matriculas" ON "Matriculas"."ID_Alumno" = "Alumnos"."ID_Alumno"  
JOIN "Expediente" ON "Expediente"."ID_Matricula" = "Matriculas"."ID_Matricula"  
JOIN "Clases" ON "Clases"."ID_Clase" = "Matriculas"."ID_Matricula"  
JOIN "Vertical" ON "Vertical"."ID_Vertical" = "Clases"."ID_Vertical"  
WHERE "Vertical"."Vertical" = 'Full Stack'  
AND "Web Dev" = 'Apto'  
AND "Front End" = 'Apto'  
AND "Back End" = 'Apto'  
AND "React" = 'Apto'  
AND "Full Stack" = 'Apto'  